In [1]:
from semantic import *


 Training
> John ate the potato
> John ate the tomato
> Mary ate the tomato

 Testing
{'Mary ate the potato': ['Mary ate the potato', 'Mary ate the tomato']}


In [ ]:
#different ideas to try
#ideas about grouping

#1 -- try grouping by looser requirements
# do analysis, every step of the way
# with example sentences

#2 -- try learning what features are not required
# a systematic way to do looser groupings